In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install ultralytics opencv-python numpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 36.5 MB/s eta 0:00:00


In [3]:
import cv2
import numpy as np
import time
from ultralytics import YOLO


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [4]:
model = YOLO("yolov8n.pt")


In [45]:
ANIMAL_CLASSES = ["cow", "goat", "sheep", "dog"]
UNAUTHORIZED_VEHICLES = ["car", "bus", "truck", "motorcycle"]
TRACTOR_CLASS = ["tractor"]

# Tractor speed threshold in km/h
SPEED_THRESHOLD = 8


In [88]:
video_path = "/content/drive/MyDrive/Systems Files/TractorVid.mp4"
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)


In [89]:
FIELD_X1, FIELD_Y1 = 100, 200
FIELD_X2, FIELD_Y2 = 600, 500


In [90]:
REAL_DISTANCE_METERS = 15
PIXEL_DISTANCE = 300
METERS_PER_PIXEL = REAL_DISTANCE_METERS / PIXEL_DISTANCE


In [91]:
tractor_positions = {}  # tractor_id -> (x, y)
tractor_times = {}      # tractor_id -> last time measured
vehicle_count = 0       # total animals + unauthorized vehicles
alerts_data = []        # For CSV logging


In [92]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    current_time = time.time()

    # Run detection
    results = model(frame)[0]

    for box in results.boxes:
        cls_id = int(box.cls[0])
        label = model.names[cls_id]

        x1, y1, x2, y2 = map(int, box.xyxy[0])
        cx = int((x1 + x2) / 2)
        cy = int((y1 + y2) / 2)

        # Check if inside crop field
        inside_field = (FIELD_X1 < cx < FIELD_X2 and FIELD_Y1 < cy < FIELD_Y2)

        # ALERTS: Animals
        if label in ANIMAL_CLASSES and inside_field:
            alert_text = f"🚨 ALERT: {label} inside crop field!"
            print(alert_text)
            alerts_data.append([current_time, label, "Animal Intrusion"])

        # ALERTS: Unauthorized Vehicles
        if label in UNAUTHORIZED_VEHICLES and inside_field:
            alert_text = f"🚨 ALERT: Unauthorized vehicle ({label}) inside crop field!"
            print(alert_text)
            alerts_data.append([current_time, label, "Unauthorized Vehicle"])

        # TRACTOR SPEED MONITORING
        if label in TRACTOR_CLASS:
            # Assign temporary ID using center coordinates (for beginners)
            tractor_id = f"{label}_{cx}_{cy}"

            if tractor_id in tractor_positions:
                prev_x, prev_y = tractor_positions[tractor_id]
                prev_time = tractor_times[tractor_id]

                pixel_distance = np.sqrt((cx - prev_x)**2 + (cy - prev_y)**2)
                distance_m = pixel_distance * METERS_PER_PIXEL
                time_diff = current_time - prev_time

                if time_diff > 0:
                    speed_mps = distance_m / time_diff
                    speed_kmph = speed_mps * 3.6

                    # Overspeed alert
                    if speed_kmph > SPEED_THRESHOLD:
                        alert_text = f"⚠️ ALERT: Tractor overspeeding! {speed_kmph:.1f} km/h"
                        print(alert_text)
                        alerts_data.append([current_time, label, "Tractor Overspeed", speed_kmph])
            else:
                speed_kmph = 0

            # Update position & time
            tractor_positions[tractor_id] = (cx, cy)
            tractor_times[tractor_id] = current_time

            # Draw tractor speed on frame
            cv2.putText(frame, f"Speed: {speed_kmph:.1f} km/h",
                        (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,255), 2)

        # Draw bounding box + label
        color = (0,255,0)
        if inside_field:
            color = (0,0,255)
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        cv2.putText(frame, label, (x1, y1-5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    # Draw crop field boundary
    cv2.rectangle(frame, (FIELD_X1, FIELD_Y1), (FIELD_X2, FIELD_Y2), (255,255,0), 2)

    # Display frame
    cv2.imshow("Smart Farm Monitoring", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


In [93]:
cap.release()
cv2.destroyAllWindows()


In [94]:
!pip install pandas
import pandas as pd

df = pd.DataFrame(alerts_data, columns=["Time", "Object", "Alert_Type", "Speed_km/h"])
df.to_csv("farm_alerts.csv", index=False)
print("✅ Alerts saved to farm_alerts.csv")


✅ Alerts saved to farm_alerts.csv


In [95]:
cap = cv2.VideoCapture(video_path)

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

out = cv2.VideoWriter('output.mp4',
                      cv2.VideoWriter_fourcc(*'mp4v'),
                      fps,
                      (frame_width, frame_height))


In [96]:
out.write(frame)


In [97]:
cap.release()
out.release()
cv2.destroyAllWindows()
print("✅ Annotated video saved as output.mp4")


✅ Annotated video saved as output.mp4


In [98]:
out = cv2.VideoWriter('output.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))
# inside loop
out.write(frame)


In [80]:
out.release()
